Importing the Required Packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Loading the dataset into the notebooks

In [2]:
# Loading the dataset
dataset = pd.read_csv("Data/new_retail_data.csv")

# Printing the top 5 rows of the dataset
dataset.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [3]:
init_shape = dataset.shape
print(f"Shape of the dataset: {init_shape}")

Shape of the dataset: (302010, 30)


# Data Cleaning and Preprocessing

1. Identification and Handling of Missing values

In [4]:
# Number of missing values in each column
print(f"Number of missing values in the dataset: \n{dataset.isnull().sum()}")

Number of missing values in the dataset: 
Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Address             315
City                248
State               281
Zipcode             340
Country             271
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Year                350
Month               273
Time                350
Total_Purchases     361
Amount              357
Total_Amount        350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64


In [5]:
# Handling the missing values according to their data types and their Distribution

# Rows that needed to be dropped
drop_rows = ["Transaction_ID", "Customer_ID", "Name",
             "Email", "Phone", "Address", "Date", "Year",
             "Month", "Time", "Product_Brand", "Order_Status"]
for col in drop_rows:
    dataset = dataset[dataset[col].notna()]

# Categorical Columns  
mode_col = ["City", "State", "Zipcode", "Country", "Gender", "Income",
            "Customer_Segment", "Total_Purchases", "Product_Category",
            "Feedback", "Shipping_Method", "Payment_Method"]
for col in mode_col:
    dataset[col] = dataset[col].fillna(dataset[col].mode()[0])

# Continous Columns and that are sensetive to outliers
median_col = ["Age", "Amount", "Total_Amount", "Ratings"]
for col in median_col:
    dataset[col] = dataset[col].fillna(dataset[col].median())

In [6]:
print(f"Shape of the dataset after handling missing values: {dataset.shape}")

Shape of the dataset after handling missing values: (298141, 30)


In [7]:
print(f"Number of the rows that are removed: {init_shape[0] - dataset.shape[0]}")

Number of the rows that are removed: 3869


In [8]:
# Data after handling missing values
dataset.isnull().sum()

Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Year                0
Month               0
Time                0
Total_Purchases     0
Amount              0
Total_Amount        0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
dtype: int64

2. Error Correction and Inconsistencies

In [9]:
print(f"Name of the columns: {dataset.columns.values}")

Name of the columns: ['Transaction_ID' 'Customer_ID' 'Name' 'Email' 'Phone' 'Address' 'City'
 'State' 'Zipcode' 'Country' 'Age' 'Gender' 'Income' 'Customer_Segment'
 'Date' 'Year' 'Month' 'Time' 'Total_Purchases' 'Amount' 'Total_Amount'
 'Product_Category' 'Product_Brand' 'Product_Type' 'Feedback'
 'Shipping_Method' 'Payment_Method' 'Order_Status' 'Ratings' 'products']


Changing the name of few columns to make them meaningful

In [10]:
dataset.rename(columns = {
    "Total_Purchases": "Quantity",
    "Amount": "Price_per_unit"
}, inplace = True)

In [11]:
dataset.describe()

,Transaction_ID,Customer_ID,Phone,Zipcode,Age,Year,Quantity,Price_per_unit,Total_Amount,Ratings
count,2.981410e+05,298141.000000,2.981410e+05,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000
mean,5.494940e+06,55007.072902,5.500495e+09,50312.113359,35.478498,2023.165090,5.359431,255.194433,1367.431405,3.162618
std,2.596053e+06,26005.990411,2.595721e+09,28964.029875,15.015371,0.371262,2.866482,141.315801,1128.452562,1.320216
min,1.000007e+06,10000.000000,1.000049e+09,501.000000,18.000000,2023.000000,1.000000,10.000219,10.003750,1.000000
25%,3.246869e+06,32469.000000,3.254198e+09,25446.000000,22.000000,2023.000000,3.000000,133.032154,439.376483,2.000000
50%,5.497883e+06,55020.000000,5.504195e+09,50652.000000,32.000000,2023.000000,5.000000,255.497589,1041.252907,3.000000
75%,7.739315e+06,77506.000000,7.748878e+09,75212.000000,46.000000,2023.000000,8.000000,377.573261,2027.455133,4.000000
max,9.999995e+06,99999.000000,9.999996e+09,99949.000000,70.000000,2024.000000,10.000000,499.997911,4999.625796,5.000000


The Mean and Median of all the Numerical columns are almost identical (except: Total_Amount), it means:
- Symmetry in Data Distribution: It indicates that the data is likely symmetrically distributed around the central value.
- Lack of Skewness: it suggests that the distribution has little to no skew.
- Robustness to Outliers: It implies that there are no extreme outliers.

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298141 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    298141 non-null  float64
 1   Customer_ID       298141 non-null  float64
 2   Name              298141 non-null  object 
 3   Email             298141 non-null  object 
 4   Phone             298141 non-null  float64
 5   Address           298141 non-null  object 
 6   City              298141 non-null  object 
 7   State             298141 non-null  object 
 8   Zipcode           298141 non-null  float64
 9   Country           298141 non-null  object 
 10  Age               298141 non-null  float64
 11  Gender            298141 non-null  object 
 12  Income            298141 non-null  object 
 13  Customer_Segment  298141 non-null  object 
 14  Date              298141 non-null  object 
 15  Year              298141 non-null  float64
 16  Month             298141 

Unique values in the Categorical columns

In [13]:
cat_cols = ["Gender", "Customer_Segment", "Product_Category", "Product_Brand",
            "Product_Type", "Feedback", "Shipping_Method", "Payment_Method", "Order_Status"]
for col in cat_cols:
    print(f"Unique values in {col}: {dataset[col].unique()}")
    print("-"*100)

Unique values in Gender: ['Male' 'Female']
----------------------------------------------------------------------------------------------------
Unique values in Customer_Segment: ['Regular' 'Premium' 'New']
----------------------------------------------------------------------------------------------------
Unique values in Product_Category: ['Clothing' 'Electronics' 'Books' 'Home Decor' 'Grocery']
----------------------------------------------------------------------------------------------------
Unique values in Product_Brand: ['Nike' 'Samsung' 'Penguin Books' 'Home Depot' 'Nestle' 'Apple' 'Zara'
 'Random House' 'Coca-Cola' 'Adidas' 'Pepsi' 'IKEA' 'HarperCollins'
 'Bed Bath & Beyond' 'Sony' 'Whirepool' 'Mitsubhisi' 'BlueStar']
----------------------------------------------------------------------------------------------------
Unique values in Product_Type: ['Shorts' 'Tablet' "Children's" 'Tools' 'Chocolate' 'Television' 'Shirt'
 'Decorations' 'Non-Fiction' 'Water' 'Snacks' 'T-shirt' '

In [14]:
data_rename = ["Product_Category", "Product_Brand", "Product_Type", "Payment_Method"]

for col in data_rename:
    dataset[col] = dataset[col].str.replace(" ", "_")

In [15]:
for col in cat_cols:
    print(f"Unique values in {col}: {dataset[col].unique()}")
    print("-"*100)

Unique values in Gender: ['Male' 'Female']
----------------------------------------------------------------------------------------------------
Unique values in Customer_Segment: ['Regular' 'Premium' 'New']
----------------------------------------------------------------------------------------------------
Unique values in Product_Category: ['Clothing' 'Electronics' 'Books' 'Home_Decor' 'Grocery']
----------------------------------------------------------------------------------------------------
Unique values in Product_Brand: ['Nike' 'Samsung' 'Penguin_Books' 'Home_Depot' 'Nestle' 'Apple' 'Zara'
 'Random_House' 'Coca-Cola' 'Adidas' 'Pepsi' 'IKEA' 'HarperCollins'
 'Bed_Bath_&_Beyond' 'Sony' 'Whirepool' 'Mitsubhisi' 'BlueStar']
----------------------------------------------------------------------------------------------------
Unique values in Product_Type: ['Shorts' 'Tablet' "Children's" 'Tools' 'Chocolate' 'Television' 'Shirt'
 'Decorations' 'Non-Fiction' 'Water' 'Snacks' 'T-shirt' '

Now, we will reduce the number of decimal places in a few columns.

#### Why We Do This Step
Reducing the number of decimal places can be important for several reasons:

- Readability: Fewer decimal places can make the data easier to read and interpret.
- Data Storage: Reducing the number of decimal places can slightly reduce the storage requirements of the dataset.
- Consistency: Ensuring that numerical data is presented consistently across different columns and datasets.

In [16]:
# Name of the columns that have multiple values after the decimal
round_col = ["Price_per_unit", "Total_Amount"]

# We will be rounding off the values to 2 decimals
for col in round_col:
    dataset[col] = dataset[col].round(2)

In [17]:
print("Number of unique columns in each column: ")
for col in dataset.columns:
    print(f"{col}: {len(dataset[col].unique())}")

Number of unique columns in each column: 
Transaction_ID: 291071
Customer_ID: 86641
Name: 158185
Email: 52740
Phone: 295810
Address: 295798
City: 130
State: 54
Zipcode: 93789
Country: 5
Age: 53
Gender: 2
Income: 3
Customer_Segment: 3
Date: 366
Year: 2
Month: 12
Time: 83579
Quantity: 10
Price_per_unit: 48879
Total_Amount: 191765
Product_Category: 5
Product_Brand: 18
Product_Type: 33
Feedback: 4
Shipping_Method: 3
Payment_Method: 4
Order_Status: 4
Ratings: 5
products: 318


3. Outliers Detection and Handling

We will be using the IQR method to find the outliers in the data.

In [18]:
# Function to return the count of outliers in each column
def iqr_val(col, threshold):
    q3 = np.percentile(col, 75)
    q1 = np.percentile(col, 25)

    IQR = q3 - q1

    lower_bound = q1 - (IQR * threshold)
    upper_bound = q3 + (IQR * threshold)

    return lower_bound, upper_bound

def detecting_outliers(data, remove_outliers = False, threshold = 1.5):
    outliers_count = {}
    new_data = data.copy()
    
    for col in new_data.columns:
        col_data = new_data[col].dropna()

        if col_data.dtype in [np.int64, np.float64]:

            lb, up = iqr_val(col = col_data, threshold=threshold)
            outliers = np.where((col_data < lb) | (col_data > up))[0]
            outliers_count[col] = len(outliers)
            
            if remove_outliers:
                new_data = new_data[(new_data[col] >= lb) & (new_data[col] <= up)]
    
    return new_data, outliers_count


In [19]:
# Names of the column which are either "int64" or "float64"
data, outliers_count_per_col = detecting_outliers(dataset, remove_outliers = True)

for col, count in outliers_count_per_col.items():
    print(f"{col}: {count}")

Transaction_ID: 0
Customer_ID: 0
Phone: 0
Zipcode: 0
Age: 0
Year: 49220
Quantity: 0
Price_per_unit: 0
Total_Amount: 3207
Ratings: 0


In [20]:
print(f"Shape of the dataset after removing the outliers: {data.shape}")

Shape of the dataset after removing the outliers: (245714, 30)


4. Data Types and Formatting

We will be changing the data types of the columns to their appropriate data types.

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245714 entries, 0 to 302008
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    245714 non-null  float64
 1   Customer_ID       245714 non-null  float64
 2   Name              245714 non-null  object 
 3   Email             245714 non-null  object 
 4   Phone             245714 non-null  float64
 5   Address           245714 non-null  object 
 6   City              245714 non-null  object 
 7   State             245714 non-null  object 
 8   Zipcode           245714 non-null  float64
 9   Country           245714 non-null  object 
 10  Age               245714 non-null  float64
 11  Gender            245714 non-null  object 
 12  Income            245714 non-null  object 
 13  Customer_Segment  245714 non-null  object 
 14  Date              245714 non-null  object 
 15  Year              245714 non-null  float64
 16  Month             245714 

In [22]:
# Changing the Zipcode, Year, Quantity and Ratings columns from "float" type to "integer".

float_col = ["Zipcode", "Year", "Quantity", "Ratings"]

for col in float_col:
    data[col] = data[col].astype("int64")

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245714 entries, 0 to 302008
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    245714 non-null  float64
 1   Customer_ID       245714 non-null  float64
 2   Name              245714 non-null  object 
 3   Email             245714 non-null  object 
 4   Phone             245714 non-null  float64
 5   Address           245714 non-null  object 
 6   City              245714 non-null  object 
 7   State             245714 non-null  object 
 8   Zipcode           245714 non-null  int64  
 9   Country           245714 non-null  object 
 10  Age               245714 non-null  float64
 11  Gender            245714 non-null  object 
 12  Income            245714 non-null  object 
 13  Customer_Segment  245714 non-null  object 
 14  Date              245714 non-null  object 
 15  Year              245714 non-null  int64  
 16  Month             245714 

Now, we have float64(6), int64(4), object(20) columns.

In [24]:
# We will be saving the preprocessed data as a new csv file for the visualization process.

file_path = "Data/cleaned_data.csv"
data.to_csv(file_path, index = False)

# Feature Engineering

1. Creation of New Features

    We will be creating 2 new features:
    - Is_Premium_Customer
    - Customer_Segment

In [25]:
data["Is_Premium_Customer"] = data["Customer_Segment"].apply(lambda x: 1 if x == "Premium" else 0)

def segment_customer(row):
    if row["Age"] < 25 and row["Income"] == "Low":
        return "Young_Low_Income"
    elif row["Age"] < 25 and row["Income"] == "High":
        return "Young_High_Income"
    elif row["Age"] >= 25 and row["Income"] == "Low":
        return "Adult_Low_Income"
    else:
        return "Adult_High_Income"

data['Customer_Segment'] = data.apply(segment_customer, axis=1)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245714 entries, 0 to 302008
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Transaction_ID       245714 non-null  float64
 1   Customer_ID          245714 non-null  float64
 2   Name                 245714 non-null  object 
 3   Email                245714 non-null  object 
 4   Phone                245714 non-null  float64
 5   Address              245714 non-null  object 
 6   City                 245714 non-null  object 
 7   State                245714 non-null  object 
 8   Zipcode              245714 non-null  int64  
 9   Country              245714 non-null  object 
 10  Age                  245714 non-null  float64
 11  Gender               245714 non-null  object 
 12  Income               245714 non-null  object 
 13  Customer_Segment     245714 non-null  object 
 14  Date                 245714 non-null  object 
 15  Year                 2

Encoding of Categorical Variables

`Country, Gender, Customer_Segment, Product_Category, Shipping_Method, Payment_Method` are Nomial data. Here, we will use one-hot encoding.

In [27]:
# All the nominal columns
nominal_col = ["Country", "Gender", "Customer_Segment", "Product_Category",
               "Shipping_Method", "Payment_Method"]

# Create dummy variables for each nominal column using pd.get_dummies()
encoded_dfs = [pd.get_dummies(data[col], prefix=col) for col in nominal_col]

# Concatenate original DataFrame and all dummy DataFrames horizontally
combined_data = pd.concat([data] + encoded_dfs, axis=1)

In [28]:
print(f"Shape of the Dataset after ohe: {combined_data.shape}")

Shape of the Dataset after ohe: (245714, 54)


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245714 entries, 0 to 302008
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Transaction_ID       245714 non-null  float64
 1   Customer_ID          245714 non-null  float64
 2   Name                 245714 non-null  object 
 3   Email                245714 non-null  object 
 4   Phone                245714 non-null  float64
 5   Address              245714 non-null  object 
 6   City                 245714 non-null  object 
 7   State                245714 non-null  object 
 8   Zipcode              245714 non-null  int64  
 9   Country              245714 non-null  object 
 10  Age                  245714 non-null  float64
 11  Gender               245714 non-null  object 
 12  Income               245714 non-null  object 
 13  Customer_Segment     245714 non-null  object 
 14  Date                 245714 non-null  object 
 15  Year                 2

Now, we will be handling Ordinal Data.
Ordinal columns are `Income, Feedback`.

In [30]:
print("Unique values in the Income col: {0}".format(data["Income"].unique()))
print("Unique values in the Feedback col: {0}".format(data["Feedback"].unique()))

Unique values in the Income col: ['Low' 'High' 'Medium']
Unique values in the Feedback col: ['Excellent' 'Average' 'Good' 'Bad']


In [31]:
income_order = {
    "Low": 0,
    "Medium": 1,
    "High": 2
}

feedback_order = {
    "Bad": 0,
    "Average": 1,
    "Good": 2,
    "Excellent": 3
}

combined_data["Income"] = combined_data["Income"].map(income_order)
combined_data["Feedback"] = combined_data["Feedback"].map(feedback_order)

In [32]:
# Dropping the columns that we don't need for training the model
col_drop = ["Transaction_ID", "Customer_ID", "Name", "Email", "Phone", "Address",
            "City", "State", "Zipcode", "Country", "Age", "Gender", "Customer_Segment",
            "Date", "Year", "Month", "Time", "Price_per_unit", "Product_Category",
            "Product_Brand", "Product_Type", "Shipping_Method", "Payment_Method",
            "Order_Status", "products"]

final_data = combined_data.drop(columns = col_drop)

In [33]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245714 entries, 0 to 302008
Data columns (total 29 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Income                              245714 non-null  int64  
 1   Quantity                            245714 non-null  int64  
 2   Total_Amount                        245714 non-null  float64
 3   Feedback                            245714 non-null  int64  
 4   Ratings                             245714 non-null  int64  
 5   Is_Premium_Customer                 245714 non-null  int64  
 6   Country_Australia                   245714 non-null  bool   
 7   Country_Canada                      245714 non-null  bool   
 8   Country_Germany                     245714 non-null  bool   
 9   Country_UK                          245714 non-null  bool   
 10  Country_USA                         245714 non-null  bool   
 11  Gender_Female                  

In [34]:
final_data.isnull().sum()

Income                                0
Quantity                              0
Total_Amount                          0
Feedback                              0
Ratings                               0
Is_Premium_Customer                   0
Country_Australia                     0
Country_Canada                        0
Country_Germany                       0
Country_UK                            0
Country_USA                           0
Gender_Female                         0
Gender_Male                           0
Customer_Segment_Adult_High_Income    0
Customer_Segment_Adult_Low_Income     0
Customer_Segment_Young_High_Income    0
Customer_Segment_Young_Low_Income     0
Product_Category_Books                0
Product_Category_Clothing             0
Product_Category_Electronics          0
Product_Category_Grocery              0
Product_Category_Home_Decor           0
Shipping_Method_Express               0
Shipping_Method_Same-Day              0
Shipping_Method_Standard              0


In [35]:
X, y = final_data.drop("Total_Amount", axis = 1), final_data.pop("Total_Amount")

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [37]:
from sklearn.linear_model import LinearRegression

In [38]:
lr = LinearRegression()

In [39]:
lr.fit(X_train, y_train)

LinearRegression()

In [40]:
pred = lr.predict(X_test)

In [41]:
from sklearn.metrics import r2_score

In [42]:
r2_score(y_test, pred)

0.40628651141536576